## Some issue with the evaluate function, so disregard the calculated recall and precision. However, the crosstab showing actuals and predictions is clear. Model tuning with recall , precision or balanced tuning - is clear in the crosstab table

In [ ]:
# !wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
# !unzip creditcardfraud
# !rm creditcardfraud.zip

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import os
import sagemaker
import boto3

In [2]:
df=pd.read_csv('creditcard.csv')

In [3]:
df.groupby('Class').count()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,284315,284315,284315,284315,284315,284315,284315,284315,284315,284315,...,284315,284315,284315,284315,284315,284315,284315,284315,284315,284315
1,492,492,492,492,492,492,492,492,492,492,...,492,492,492,492,492,492,492,492,492,492


# Cleaning steps
1. NUll removal
2. feature reduction
3. scaling

In [4]:
df.isna().sum().sum()

0

In [5]:
df.describe() # looks like its already scaled except for amount

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
scaler=StandardScaler()
df['Amount']=scaler.fit_transform(df[['Amount']])

## Train, Test Sets

In [7]:
df=df.drop(['Time'],axis=1)

In [8]:
X=df.drop(['Class'],axis=1)
y=df[['Class']]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [58]:
data_dir='data_dir'

In [ ]:
pd.concat([y_train,X_train],axis=1).to_csv(os.path.join(data_dir,'train.csv'),header=False,index=False)
pd.concat([y_test,X_test],axis=1).to_csv(os.path.join(data_dir,'test.csv'),header=False,index=False)

In [59]:
session=sagemaker.Session()
role=sagemaker.get_execution_role()
region=session.boto_region_name
bucket=session.default_bucket()
prefix='payment_fraud_detection'
image_uri=sagemaker.image_uris.retrieve(framework='linear-learner',region=region)

In [60]:
s3_train_data=session.upload_data(os.path.join(data_dir,'train.csv'),bucket=bucket,key_prefix=prefix)
s3_test_data=session.upload_data(os.path.join(data_dir,'test.csv'),bucket=bucket,key_prefix=prefix)

In [ ]:
linear_learner=sagemaker.estimator.Estimator(image_uri=image_uri,
                                             role=role,
                                             instance_count=1,
                                             instance_type='ml.m5.large',
                                             use_spot_instances=True,
                                             max_run=3600,
                                             max_wait=3600,
                                             sagemaker_session=session,
                                             hyperparameters={
                                                 'predictor_type':'binary_classifier',
                                                 'epochs':5,
                                                 'num_models':5,
                                                 'loss':'logistic'
                                             }
                                             
)

In [ ]:
# #Should test this later
# sagemaker.inputs.TransformInput()
# s3_input_test=sagemaker.s3_input(s3_data=s3_test_data,
#                                   content_type='text/csv')

In [61]:
s3_input_train=sagemaker.inputs.TrainingInput(s3_data=s3_train_data,
                                  content_type='text/csv')


In [ ]:
linear_learner.fit({'train':s3_input_train})

In [ ]:
%%time 
# deploy and create a predictor
linear_predictor = linear_learner.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [50]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [linear_predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    prediction_batches2=[[x for x in batch['predictions']] for batch in prediction_batches]
    prediction_batches=None
    test_preds = np.array([score['predicted_label'] for batch in prediction_batches2 for score in batch])
    prediction_batches2=None
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels.flatten(), test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


In [11]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

linear_predictor.serializer = CSVSerializer()
linear_predictor.deserializer = JSONDeserializer()

In [ ]:
# import numpy as np
# y=np.random.randint(-1, high=1, size=(1,29), dtype=int)
# print(y.shape)
# linear_predictor.predict(y)

In [14]:
# Disconnected session so creating predictor object from an already existing endpoint
# from sagemaker.predictor import Predictor

# linear_predictor = Predictor(endpoint_name='linear-learner-2021-05-02-14-35-19-535', sagemaker_session=session)
# linear_predictor.serializer = CSVSerializer()
# linear_predictor.deserializer = JSONDeserializer()

In [51]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   np.array(X_test).astype('float32'), 
                   np.array(y_test), 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)      0    1
actual (row)                
0                 93804   34
1                    25  124

Recall:     0.002
Precision:  0.002
Accuracy:   0.997



In [54]:
boto3.client('sagemaker').delete_endpoint(EndpointName=linear_predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'ResponseMetadata': {'RequestId': 'a25dd07b-56b0-4d24-9a11-7e49f317c466',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a25dd07b-56b0-4d24-9a11-7e49f317c466',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 02 May 2021 16:15:44 GMT'},
  'RetryAttempts': 0}}

# Adjust for class imbalance and a target recall

In [62]:
linear_learner=sagemaker.estimator.Estimator(image_uri=image_uri,
                                             role=role,
                                             instance_count=1,
                                             instance_type='ml.m5.large',
                                             use_spot_instances=True,
                                             max_run=3600,
                                             max_wait=3600,
                                             sagemaker_session=session,
                                             hyperparameters={
                                                 'predictor_type':'binary_classifier',
                                                 'epochs':5,
                                                 'num_models':5,
                                                 'loss':'logistic',
                                                 'binary_classifier_model_selection_criteria':'precision_at_target_recall',
                                                 'target_recall':0.9,
                                                 'positive_example_weight_mult':'balanced'
                                             }
                                             
)

In [63]:
linear_learner.fit({'train':s3_input_train})

2021-05-02 16:17:20 Starting - Starting the training job...
2021-05-02 16:17:47 Starting - Launching requested ML instancesProfilerReport-1619972239: InProgress
......
2021-05-02 16:18:47 Starting - Preparing the instances for training.........
2021-05-02 16:20:07 Downloading - Downloading input data...
2021-05-02 16:20:48 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/02/2021 16:20:47 INFO 140135845263168] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

#metrics {"StartTime": 1619972460.2543287, "EndTime": 1619972460.2543836, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3260033696626362, "count": 1, "min": 0.3260033696626362, "max": 0.3260033696626362}}}

#metrics {"StartTime": 1619972460.254487, "EndTime": 1619972460.2545362, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3515915467513235, "count": 1, "min": 0.3515915467513235, "max": 0.3515915467513235}}}

#metrics {"StartTime": 1619972460.254618, "EndTime": 1619972460.2546315, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 2}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.324260

[05/02/2021 16:21:15 INFO 140135845263168] #train_score (algo-1) : ('binary_classification_weighted_cross_entropy_objective', 0.29129383753660704)
[05/02/2021 16:21:15 INFO 140135845263168] #train_score (algo-1) : ('binary_classification_accuracy', 0.9834398909967509)
[05/02/2021 16:21:15 INFO 140135845263168] #train_score (algo-1) : ('binary_f_1.000', 0.16357861302276336)
[05/02/2021 16:21:15 INFO 140135845263168] #train_score (algo-1) : ('precision', 0.08995633187772925)
[05/02/2021 16:21:15 INFO 140135845263168] #train_score (algo-1) : ('recall', 0.9008746355685131)
[05/02/2021 16:21:15 INFO 140135845263168] #train_score (algo-1) : ('roc_auc_score', 0.9846922665884793)
[05/02/2021 16:21:15 INFO 140135845263168] #quality_metric: host=algo-1, train binary_classification_weighted_cross_entropy_objective <loss>=0.29129383753660704
[05/02/2021 16:21:15 INFO 140135845263168] #quality_metric: host=algo-1, train binary_classification_accuracy <score>=0.9834398909967509
[05/02/2021 16:21:15 

In [64]:
# deploy and create a predictor
linear_predictor = linear_learner.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------------!

In [65]:
linear_predictor.serializer = CSVSerializer()
linear_predictor.deserializer = JSONDeserializer()

In [66]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   np.array(X_test).astype('float32'), 
                   np.array(y_test), 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)      0     1
actual (row)                 
0                 92314  1524
1                    15   134

Recall:     0.018
Precision:  0.002
Accuracy:   0.981



In [67]:
boto3.client('sagemaker').delete_endpoint(EndpointName=linear_predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'ResponseMetadata': {'RequestId': 'aec550aa-d6c3-4907-8c06-90cfc7654ca4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aec550aa-d6c3-4907-8c06-90cfc7654ca4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 02 May 2021 16:32:39 GMT'},
  'RetryAttempts': 0}}

# Adjust for class imbalance and a target precision

In [68]:
linear_learner=sagemaker.estimator.Estimator(image_uri=image_uri,
                                             role=role,
                                             instance_count=1,
                                             instance_type='ml.m5.large',
                                             use_spot_instances=True,
                                             max_run=3600,
                                             max_wait=3600,
                                             sagemaker_session=session,
                                             hyperparameters={
                                                 'predictor_type':'binary_classifier',
                                                 'epochs':5,
                                                 'num_models':5,
                                                 'loss':'logistic',
                                                 'binary_classifier_model_selection_criteria':'recall_at_target_precision',
                                                 'target_precision':0.9,
                                                 'positive_example_weight_mult':'balanced'
                                             }
                                             
)

In [69]:
linear_learner.fit({'train':s3_input_train})

2021-05-02 16:32:40 Starting - Starting the training job...
2021-05-02 16:33:03 Starting - Launching requested ML instancesProfilerReport-1619973159: InProgress
.........
2021-05-02 16:34:23 Starting - Preparing the instances for training......
2021-05-02 16:35:25 Downloading - Downloading input data...
2021-05-02 16:36:04 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/02/2021 16:36:23 INFO 140358224295744] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer':

#metrics {"StartTime": 1619973401.8050601, "EndTime": 1619973401.805091, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3065602360374049, "count": 1, "min": 0.3065602360374049, "max": 0.3065602360374049}}}

#metrics {"StartTime": 1619973401.8051527, "EndTime": 1619973401.8051655, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3160341179697137, "count": 1, "min": 0.3160341179697137, "max": 0.3160341179697137}}}

#metrics {"StartTime": 1619973401.8052056, "EndTime": 1619973401.8052325, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 2}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.30518

In [70]:
# deploy and create a predictor
linear_predictor = linear_learner.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------------!

In [71]:
linear_predictor.serializer = CSVSerializer()
linear_predictor.deserializer = JSONDeserializer()

In [72]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   np.array(X_test).astype('float32'), 
                   np.array(y_test), 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)      0   1
actual (row)               
0                 93833   5
1                   103  46

Recall:     0.001
Precision:  0.002
Accuracy:   0.998



In [73]:
boto3.client('sagemaker').delete_endpoint(EndpointName=linear_predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'ResponseMetadata': {'RequestId': 'a67bcdf7-e1bb-4367-9571-99c21dd85874',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a67bcdf7-e1bb-4367-9571-99c21dd85874',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 02 May 2021 16:47:50 GMT'},
  'RetryAttempts': 0}}